In [23]:
import pandas as pd
import sqlite3

#read webscrapping data
df_atm = pd.read_csv('/Users/dunghoang/GitHub/SupplyChain/csv_files/atm.csv')
df_company_details = pd.read_csv('/Users/dunghoang/GitHub/SupplyChain/csv_files/company_details.csv')
df_reviews = pd.read_csv('/Users/dunghoang/GitHub/SupplyChain/csv_files/reviews.csv')

# Connect to the SQLite database or create a new one
conn = sqlite3.connect('PostgreSQL 15')

# Create a cursor object to interact with the database
cursor = conn.cursor()

In [24]:
# ATM TABLE
#create atm table
create_table_atm = '''
CREATE TABLE IF NOT EXISTS atm (
    company_id INTEGER,
    company_name TEXT,
    trustscore INTEGER,
    total_reviews INTEGER,
    domain TEXT
)
'''
cursor.execute(create_table_atm)

# Insert data into the table using pandas DataFrame
df_atm.to_sql('atm', conn, if_exists='append', index=False)

# Commit the changes to the database
conn.commit()

# Read the table into a pandas DataFrame
df_preview = pd.read_sql_query('SELECT * FROM atm', conn)

# Display the DataFrame
print(df_preview)

    company_id                                    company_name  trustscore  \
0          110                          Evergreen Credit Union         4.9   
1          111                      Liberty First Credit Union         4.8   
2          112                              Crypto Dispensers          2.3   
3          113                                         Swadesh         4.4   
4          114            Heritage Valley Federal Credit Union         4.1   
5          115                                  The Payment HQ         3.8   
6          116                                        ASDFTEST         3.7   
7          117  WPC Merchant Services & Credit Card Processing         3.7   
8          118                                Kryptominerstech         3.7   
9          119                           Meriwest Credit Union         3.5   
10        1111                               cryptobaseatm.com         3.0   
11        1112                                        NorthOne  

In [25]:
# COMPANY_DETAILS TABLE

#create company_details table
create_table_company_details = '''
CREATE TABLE IF NOT EXISTS company_details (
    company_name TEXT,
    rating_class INT,
    star_5 TEXT,
    star_4 TEXT,
    star_3 TEXT,
    star_2 TEXT,
    star_1 TEXT,
    'total_reviews' INT
)
'''
cursor.execute(create_table_company_details)

# Insert data into the table using pandas DataFrame
df_company_details.to_sql('company_details', conn, if_exists='append', index=False)

# Commit the changes to the database
conn.commit()

# Read the table into a pandas DataFrame
df_preview = pd.read_sql_query('SELECT * FROM company_details', conn)

# Display the DataFrame
print(df_preview)

                                      company_name rating_class star_5 star_4  \
0             Heritage Valley Federal Credit Union        Great    82%     0%   
1                                         ASDFTEST      Average   100%     0%   
2                              Coinhub Bitcoin ATM      Average     0%     0%   
3   WPC Merchant Services & Credit Card Processing      Average   100%     0%   
4                                     Cash Express      Average    25%    17%   
5                                         PNC Bank          Bad     5%     1%   
6                       Liberty First Credit Union    Excellent    93%     3%   
7                            Meriwest Credit Union      Average    62%    13%   
8                                         NorthOne      Average    40%     7%   
9                                   The Payment HQ        Great   100%     0%   
10                                   Asicminertech         Poor    14%     7%   
11                          

In [26]:
# REVEWS TABLE

#create reviews table
create_table_reviews = '''
CREATE TABLE IF NOT EXISTS reviews (
    company_name TEXT,
    review_star TEXT,
    review_title TEXT,
    reviewer_name TEXT,
    review_text TEXT,
    experience_date TEXT,
    review_date DATE,
    reply_date TEXT,
    reply_text TEXT
)
'''
cursor.execute(create_table_reviews)

# Insert data into the table using pandas DataFrame
df_reviews.to_sql('reviews', conn, if_exists='append', index=False)

# Commit the changes to the database
conn.commit()

# Read the table into a pandas DataFrame
df_preview = pd.read_sql_query('SELECT * FROM reviews', conn)

# Display the DataFrame
print(df_preview)

     company_name review_star                             review_title  \
0         Swadesh  5 out of 5               International transactions   
1         Swadesh  5 out of 5  Swadesh is a good app safely and truly…   
2         Swadesh  5 out of 5                                  Because   
3         Swadesh  5 out of 5                    Cross border payments   
4         Swadesh  5 out of 5                            Cool Benefits   
...           ...         ...                                      ...   
5311      Swadesh  5 out of 5                        Banking in the US   
5312      Swadesh  5 out of 5               Made for Indians in the US   
5313      Swadesh  5 out of 5                              Easy to use   
5314      Swadesh  5 out of 5             Working experience is 4years   
5315      Swadesh  5 out of 5                           Swadeshi Feels   

                  reviewer_name  \
0                      Eric Low   
1                  Hamza masood   
2     

In [34]:
# JOIN INTO COMPANIES_WITHID TABLE

#create reviews table
create_table_atmcompanies_withids= '''
CREATE TABLE IF NOT EXISTS atmcompanies_withid (
    company_id INTEGER PRIMARY KEY,
    company_name TEXT,
    trustscore INTEGER,
    total_reviews INTEGER,
    domain TEXT,
    rating_class INTEGER,
    star_5 TEXT,
    star_4 TEXT,
    star_3 TEXT,
    star_2 TEXT,
    star_1 TEXT
)
'''
conn.execute(create_table_atmcompanies_withids)
cursor.execute(create_table_atmcompanies_withids)

# SQL code for the INSERT operation
insert_sql = '''
INSERT INTO atmcompanies_withid (company_id, company_name, trustscore, total_reviews, domain, rating_class, star_5, star_4, star_3, star_2, star_1)
SELECT
    atm.company_id,
    atm.company_name,
    atm.trustscore,
    atm.total_reviews,
    atm.domain,
    cd.rating_class,
    cd.star_5,
    cd.star_4,
    cd.star_3,
    cd.star_2,
    cd.star_1
FROM atm AS atm
INNER JOIN company_details AS cd ON atm.company_name = cd.company_name
'''

# Commit the changes to the database
conn.commit()

# Read the table into a pandas DataFrame
df_preview = pd.read_sql_query('SELECT * FROM atmcompanies_withid', conn)

# Display the DataFrame
print(df_preview)

# Export the DataFrame to a CSV file
#df_preview.to_csv('atmcompanies_withid.csv', index=False)

#print("CSV file exported successfully.")

Empty DataFrame
Columns: [company_id, company_name, trustscore, total_reviews, domain, rating_class, star_5, star_4, star_3, star_2, star_1]
Index: []


In [28]:
# GENERATE REVIEW IDS

# Generate review IDs using pandas DataFrame method
df_reviews_withid = df_reviews.reset_index().rename(columns={'index': 'review_id'})

print(df_reviews_withid)

      review_id company_name review_star  \
0             0      Swadesh  5 out of 5   
1             1      Swadesh  5 out of 5   
2             2      Swadesh  5 out of 5   
3             3      Swadesh  5 out of 5   
4             4      Swadesh  5 out of 5   
...         ...          ...         ...   
1767       1767      Swadesh  5 out of 5   
1768       1768      Swadesh  5 out of 5   
1769       1769      Swadesh  5 out of 5   
1770       1770      Swadesh  5 out of 5   
1771       1771      Swadesh  5 out of 5   

                                 review_title              reviewer_name  \
0                  International transactions                   Eric Low   
1     Swadesh is a good app safely and truly…               Hamza masood   
2                                     Because            Abdulaziz Aliye   
3                       Cross border payments  Nagaraju Doddasomanahalli   
4                               Cool Benefits           Vignesh N Shetty   
...            

In [31]:
import pandas as pd
import sqlite3

# Connect to the SQLite database or use the existing connection
conn = sqlite3.connect('PostgreSQL 15')

# Read the 'reviews_withid' and 'atmcompanies_withid' tables into pandas DataFrames
df_reviews_withid = pd.read_sql_query('SELECT * FROM reviews_withid', conn)
df_atmcompanies_withid = pd.read_sql_query('SELECT * FROM atmcompanies_withid', conn)

# Check if the DataFrames are loaded correctly
print("reviews_withid DataFrame:")
print(df_reviews_withid)

print("\natmcompanies_withid DataFrame:")
print(df_atmcompanies_withid)

# Close the connection
conn.close()

DatabaseError: Execution failed on sql 'SELECT * FROM reviews_withid': no such table: reviews_withid

In [33]:
#CREATE TABLE WITH REVIEW IDS

# Create the 'reviews_withid' table
create_table_reviews_withid = '''
CREATE TABLE IF NOT EXISTS reviews_withid (
    review_id INTEGER PRIMARY KEY,
    company_name TEXT,
    review_star TEXT,
    review_title TEXT,
    reviewer_name TEXT,
    review_text TEXT,
    experience_date TEXT,
    review_date DATE,
    reply_date TEXT,
    reply_text TEXT
)
'''
conn.execute(create_table_reviews_withid)

# Insert data into the 'reviews_withid' table using pandas DataFrame
df_reviews_withid.to_sql('reviews_withid', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

# Read the 'reviews_withid' table into a pandas DataFrame
df_preview_reviews_withid = pd.read_sql_query('SELECT * FROM reviews_withid', conn)

# write to new csv file: 
df_preview_reviews_withid.to_csv('/Users/dunghoang/GitHub/SupplyChain/csv_files/reviews_withallids.csv', index=False)
# Display the DataFrame
print(df_preview_reviews_withid)

      review_id company_name review_star  \
0             0      Swadesh  5 out of 5   
1             1      Swadesh  5 out of 5   
2             2      Swadesh  5 out of 5   
3             3      Swadesh  5 out of 5   
4             4      Swadesh  5 out of 5   
...         ...          ...         ...   
1767       1767      Swadesh  5 out of 5   
1768       1768      Swadesh  5 out of 5   
1769       1769      Swadesh  5 out of 5   
1770       1770      Swadesh  5 out of 5   
1771       1771      Swadesh  5 out of 5   

                                 review_title              reviewer_name  \
0                  International transactions                   Eric Low   
1     Swadesh is a good app safely and truly…               Hamza masood   
2                                     Because            Abdulaziz Aliye   
3                       Cross border payments  Nagaraju Doddasomanahalli   
4                               Cool Benefits           Vignesh N Shetty   
...            

In [ ]:

#CREATE TABLE WITH REVIEW AND JOIN COMPANY IDS
# Create the 'reviews_withallids' table

create_table_reviews_withallids = '''
CREATE TABLE IF NOT EXISTS reviews_withallids (
    review_id INTEGER PRIMARY KEY,
    company_id INT, 
    company_name TEXT,
    review_star TEXT,
    review_title TEXT,
    reviewer_name TEXT,
    review_text TEXT,
    experience_date TEXT,
    review_date DATE,
    reply_date TEXT,
    reply_text TEXT
)
'''

cursor.execute(create_table_reviews_withallids)
conn.commit()

# SQL code for the INSERT operation
insert_into_review_sql = '''
INSERT INTO reviews_withallids (review_id, company_id, company_name, review_star, review_title, reviewer_name, review_text, experience_date, review_date, reply_date, reply_text)
SELECT
    r.review_id,
    a.company_id,
    r.company_name,
    r.review_star,
    r.review_title,
    r.reviewer_name,
    r.review_text,
    r.experience_date,
    r.review_date,
    r.reply_date,
    r.reply_text
FROM reviews_withid AS r
JOIN atmcompanies_withid AS a ON r.company_name = a.company_name
'''

cursor.execute(insert_into_review_sql)
conn.commit()

In [30]:
# READ AND PRINT THE TABLE REVIEWS WITH ALL IDS

# Read the 'reviews_withid' table into a pandas DataFrame
df_preview_reviews_withallids = pd.read_sql_query('SELECT * FROM reviews_withallids', conn)

# Display the DataFrame
print(df_preview_reviews_withallids)

# Export the DataFrame to a CSV file
df_preview_reviews_withallids.to_csv('reviews_withallids.csv', index=False)

print("CSV file exported successfully.")

DatabaseError: Execution failed on sql 'SELECT * FROM reviews_withallids': no such table: reviews_withallids